# Plot pentamer formation FPT comparison plots

Loads data computed in scripts and generates the plots (used in paper) comparing first passage times (FPTs) distribution between the benchmark model and MSM/RD for the pentameric ring formation example. The pentamer formation example consists of five equal patchy particles, each with two patches. The interaction potential is given by the patchy particle angular 2. it can be modified to obtain the FPT distributions of th formation of trimeric and tetrameric ring molecules as well.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import msmrd2.tools.analysis as analysisTools
import timeit
from matplotlib import rc
rc('text', usetex=True)

In [ ]:
# Main parameters
numparticles = 5
boxsize = 6 #2.5 #10 #8 #6 #5 
D = 1.0
Drot = 1.0
lagtime = 40 # 100
rootDirectory = '../../data/pentamer/first_passage_times/'

In [ ]:
# Color definitions
c1 = '#ff3300' #'xkcd:bright orange' # 'darkorange' #'C1' # '#ff3300' # for benchmark
c2 = '#0099ff' #'royalblue' #'royalblue' # 'C0' # '#0099ff' # for MSM/RD
c1alt = 'xkcd:bright orange'
c2alt = 'royalblue'
# Alpha transparency definition
a1 = 0.2 #0.2
a2 = 0.7 #0.9 #0.7
# referenceState = 'trimeric-loop' or 'tetrameric-loop' or 'pentameric-loop' (sometimes 'pentamer' for the last one)
referenceState = 'pentameric-loop' # change to evaluate trimeric/tetrameric or pentameric formations

## Load FPT for trimer formation data (bechmark and MSMRD)

In [ ]:
# Load benchmark FPT data computed by script
numTrajs = 600000
filename1 = rootDirectory + 'pentamerFPTs_trajs' + str(numTrajs) + '_boxsize' + str(boxsize) + '_final.xyz'
fpts = []

with open(filename1) as file:
    for line in file:
        state, time = line.split(" ")
        if state == referenceState:
            time = float(time)
            fpts.append(time)
            if len(fpts) >= 5000:
                break

# Normalize weights
weights = np.ones_like(fpts)/float(len(fpts))
print('The number of trajectories ending in ' + referenceState + 's is: ', len(fpts))

In [ ]:
# Load FPT data obtained with MSMRD integrator
numTrajs = 25000 #20000 #25000 
filename = rootDirectory + 'MSMRDpentamerFPTs_trajs' + str(numTrajs) \
                         + '_lagt' + str(lagtime) +  '_boxsize' + str(boxsize) + '_final.xyz'
MSMRDfpts = []
i=1
with open(filename) as file:
    for line in file:
        state, time = line.split(" ")
        if state == referenceState:
            time = float(time)
            MSMRDfpts.append(time)
            if len(MSMRDfpts) >= 5000:
                break
            
# Normalize weights
#MSMRDfpts = MSMRDfpts[4000:9000]
weightsMSMRD = np.ones_like(MSMRDfpts)/float(len(MSMRDfpts))
print('The number of MSMRD trajectories ending in ' + referenceState + 's is: ', len(MSMRDfpts))

### Calculation of FPTs and comparisons plots

In [ ]:
# Compuate mean and standard deviation of bootstrapped samples of benchmark
calculateRates = True
numBootsrapSamples = 2000
mfpt, fptstd = analysisTools.bootstrapping(fpts, numBootsrapSamples)
print("Raw MFPTs: ", np.array(fpts).mean())
print("Bootstrapping mean and std (A): ", mfpt, fptstd)
# Compuate mean and standard deviation of bootstrapped samples of MSMRD simulation
MSMRDmfpt, MSMRDfptstd = analysisTools.bootstrapping(MSMRDfpts, numBootsrapSamples)
print("Raw MSMRD MFPTs: ",np.array(MSMRDfpts).mean())
print("MSMRD bootstrapping mean and std: ", MSMRDmfpt, MSMRDfptstd)
print("Trajectory lengths")
if calculateRates:
    mkon = 1.0/mfpt
    MSMRDmkon = 1.0/MSMRDmfpt
    konstd = fptstd/(mfpt*mfpt)
    MSMRDkonstd = MSMRDfptstd/(MSMRDmfpt*MSMRDmfpt)
    errorKon = np.abs(mkon - MSMRDmkon)
    relErrorKon = errorKon/mkon
    print("On rates:")
    print("Bootstrapping mean and std:", mkon, konstd)
    print("Bootstrapping MSMRD mean and std:", MSMRDmkon, MSMRDkonstd)
    print("Error and relative error:", errorKon, relErrorKon)

In [ ]:
# PLOT FOR PAPER #
# Configure figure
fig, ax1 = plt.subplots(nrows=1, figsize=(7,5))
fig.subplots_adjust(hspace=0)
fsize = 18
plt.rcParams['xtick.labelsize']=fsize
plt.rcParams['ytick.labelsize']=fsize
# Set outer common axes titles
ax0 = fig.add_subplot(111, frameon=False)
ax0.set_xlabel(r'$\mathrm{time}$', fontsize=fsize + 4)
ax0.set_ylabel(r'$\mathrm{FPTs \ distribution}$', labelpad=17, fontsize=fsize + 4)
ax0.set_xticks([0])
ax0.set_yticks([0])
#hbins = range(0, 500,15)
#hbins = range(0, 400,12)
hbins = range(0, 200,6)
#hbins = range(0, 300,9)
#hbins = range(0, 100,3)
#hbins = range(0, 33,1)


# Plot A to unbound histogram
#legendTitle = r'Trimeric ring'
#legendTitle = r'Tetrameric ring'
legendTitle = r'Pentameric ring'

ax1.hist(fpts, bins = hbins, alpha=a1, color=c1, weights = weights );
ax1.hist(fpts, bins = hbins, alpha=a2, edgecolor=c1, lw=2, histtype='step', 
         facecolor = 'None', weights = weights, label =r'$\mathrm{Benchmark}$');
ax1.hist(MSMRDfpts, bins = hbins, alpha=a1, color=c2, weights = weightsMSMRD);
ax1.hist(MSMRDfpts, bins = hbins, alpha=a2, edgecolor=c2, lw=2, histtype='step', 
         weights = weightsMSMRD, label =r'$\mathrm{MSM/RD}$');
ax1.legend(fontsize=fsize, title=legendTitle, title_fontsize = fsize, fancybox=True)
#ax1.text(45, 0.125, r'$\tau = {:.2f} \pm {:.2f}$'.format(mfpt, fptstd), fontsize=fsize, color=c1)
#ax1.text(55, 0.11, r'$\tau = {:.2f} \pm {:.2f}$'.format(MSMRDmfpt, MSMRDfptstd), fontsize=fsize, color=c2alt)
ax1.text(48, 0.11, r'$\tau = {:.2f} \pm {:.2f}$'.format(mfpt, fptstd), fontsize=fsize, color=c1)
ax1.text(55, 0.10, r'$\tau = {:.2f} \pm {:.2f}$'.format(MSMRDmfpt, MSMRDfptstd), fontsize=fsize, color=c2alt)
ax1.set_yticks(np.arange(0, 0.18, step=0.1));
#ax1.set_xlim([0,33])
#ax1.set_ylim([0,0.18])
#ax1.set_xlim([0,400])
ax1.set_xlim([0,200])

#plt.savefig('trimerFPTs.pdf', bbox_inches='tight')
#plt.savefig('trimerFPTs.svg', bbox_inches='tight')
#plt.savefig('tetramerFPTs.pdf', bbox_inches='tight')
#plt.savefig('tetramerFPTs.svg', bbox_inches='tight')
#plt.savefig('pentamerFPTs.pdf', bbox_inches='tight')
#plt.savefig('pentamerFPTs.svg', bbox_inches='tight')

 ## Load FPTs for pentameric $k_\text{on}$ plot

In [ ]:
# Load FPT data obtained with MSMRD integrator
numTrajs = 20000 #20000 #25000 
numValues = 1000
boxsizes = [6,7,8,9,10,11]
#boxsizes = [6,7,8,9,10]
numBoxes = len(boxsizes)
fpt = [[] for i in range(numBoxes)]
MSMRDfpt = [[] for i in range(numBoxes)]
for i, boxlen in enumerate(boxsizes):
    filename1 = rootDirectory + 'pentamerFPTs_boxsize' + str(boxlen) + '.xyz'
    filename2 = rootDirectory + 'MSMRDpentamerFPTs_trajs' + str(numTrajs) \
                         + '_lagt' + str(lagtime) +  '_boxsize' + str(boxlen) + '.xyz'
    with open(filename1) as file:
        for line in file:
            state, time = line.split(" ")
            fpt[i].append(float(time))
            if len(fpt[i]) >= numValues:
                break
    with open(filename2) as file:
        for line in file:
            state, time = line.split(" ")
            if state == 'pentameric-loop':
                MSMRDfpt[i].append(float(time))
                if len(MSMRDfpt[i]) >= numValues:
                    break
    print(boxlen, len(fpt[i]), len(MSMRDfpt[i]))

In [ ]:
### Calculate meand and bootstrapping error
numBootsrapSamples = 100
meanKon = np.zeros(numBoxes)
meanKonMSMRD = np.zeros(numBoxes)
stdDevKon = np.zeros(numBoxes)
stdDevKonMSMRD = np.zeros(numBoxes)
errorKon = np.zeros(numBoxes)
errorKonMSMRD = np.zeros(numBoxes)
diffKon = np.zeros(numBoxes)
relErrorKon = np.zeros(numBoxes)
for i, boxlen in enumerate(boxsizes):
    # Calculate bootstrapping samples of mean first passage times
    mfpt, fptStdDev = analysisTools.bootstrapping(fpt[i], numBootsrapSamples)
    MSMRDmfpt, MSMRDfptStdDev = analysisTools.bootstrapping(MSMRDfpt[i], numBootsrapSamples)
    # Calculate on rate corresponding to bootstrapping samples
    meanKon[i], stdDevKon[i] = 1.0/mfpt, fptStdDev/(mfpt*mfpt)
    meanKonMSMRD[i], stdDevKonMSMRD[i] = 1.0/MSMRDmfpt, MSMRDfptStdDev/(MSMRDmfpt*MSMRDmfpt)
    # Percentual errors
    errorKon[i] = stdDevKon[i] #1.96*stdDevKon[i]/numBootsrapSamples
    errorKonMSMRD[i] = stdDevKonMSMRD[i] #1.96*stdDevKonMSMRD[i]/numBootsrapSamples
    
    diffKon[i] = np.abs(meanKon[i] - meanKonMSMRD[i])
    relErrorKon[i] = diffKon[i]/meanKon[i]
    print("On rates for boxsize:",boxlen)
    print("  Bootstrapping mean and std:", meanKon[i], stdDevKon[i])
    print("  Bootstrapping MSMRD mean and std:", meanKonMSMRD[i], stdDevKonMSMRD[i])
    print("  Error and relative error:", diffKon[i], relErrorKon[i])

In [ ]:
# Plot on rates vs concentration:
fig = plt.figure()
fsize = 16
plt.rcParams['xtick.labelsize']=fsize
plt.rcParams['ytick.labelsize']=fsize
ax = fig.add_subplot(111)
linewidth = 2
concentrations = 5/pow(5*np.array(boxsizes),3)
# Assuming moleculas have a diameter of 5nm, the actual boxsize is 5*boxisze nm,
# lets convert to Molar concentration
avogadro = 6.022E23
oneLiter2nm = 1.0E24
concentrationsMolar = concentrations*oneLiter2nm/avogadro

# Plots with shaded regions as error bars
ax.plot(concentrationsMolar, meanKon, marker='o', markersize=6, linewidth=linewidth , color='#ff5b00', 
        label=r'$\mathrm{Benchmark}$')
ax.fill_between(concentrationsMolar, meanKon-errorKon, meanKon+errorKon, alpha=0.3, edgecolor='#ff5b00', 
                facecolor='#ff5b00')

# Traditional error bar plots
ax.errorbar(concentrationsMolar, meanKonMSMRD, yerr=errorKonMSMRD, marker='o', markersize=6, linestyle='--',
            elinewidth=2, linewidth=linewidth, capsize=5, color='#4169e1', label=r'$\mathrm{MSM/RD}$')

ax.set_yscale('log')
#ax.set_xlabel(r'$\mathrm{Concentration}$ ($1/\mathrm{Volume}$)', fontsize = fsize+4)
ax.set_xlabel(r'$\mathrm{Concentration}$ ($M$)', fontsize = fsize+4)
ax.set_ylabel(r'$\mathrm{log}(k_\mathrm{on}$)', fontsize = fsize+6)
#ax.set_yticks([0.001,.01])
#ax.set_ylim([0,0.12])
ax.ticklabel_format(axis="x", style="sci",scilimits=(0,0))
ax.legend(fontsize=fsize, title_fontsize = fsize, fancybox=True)
#plt.savefig('kon_pentamer.pdf', bbox_inches='tight')